# How-To, First Steps

## Load package and create basic options

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from guaranteed.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [2]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [3]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <guaranteed.finance.derivatives.AmericanOption object at 0x10989dd90>
Option 2: <guaranteed.finance.derivatives.EuropeanOption object at 0x10989dd30>
Option 3: <guaranteed.finance.derivatives.BermudanOption object at 0x102b9a3a0>
Option 4: <guaranteed.finance.derivatives.AmericanOption object at 0x102b9a2b0>


In [4]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `guaranteed.pricing`

In [8]:
from guaranteed.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [10]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [11]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0896 sec (CPU 0.0896 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 19/90 (21.11%)
iter = 58/90 (64.44%)
iter = 75/90 (83.33%)
t = 3
iter = 8/67 (11.94%)
iter = 22/67 (32.84%)
t = 2
iter = 20/45 (44.44%)
iter = 33/45 (73.33%)
t = 1
t = 0
Computing value function in intermediate points in time: 0.5089 sec (CPU 0.5089 sec)
Solving the problem: 0.5992 sec (CPU 0.5992 sec)
Value: 5.327786420219619


Let's play around and change Lattice step

In [12]:
pm1.lattice = Lattice(delta=[.1])
sol2 = solver.solve(pm1)
print('Value: {0}'.format(sol2['Vf'][0][0]))

Precalculating points for value function evaluation: 0.7393 sec (CPU 0.7393 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 8/818 (0.98%)
iter = 46/818 (5.62%)
iter = 123/818 (15.04%)
iter = 188/818 (22.98%)
iter = 323/818 (39.49%)
iter = 340/818 (41.56%)
iter = 352/818 (43.03%)
iter = 370/818 (45.23%)
iter = 387/818 (47.31%)
iter = 388/818 (47.43%)
iter = 452/818 (55.26%)
iter = 467/818 (57.09%)
iter = 485/818 (59.29%)
iter = 511/818 (62.47%)
iter = 585/818 (71.52%)
iter = 776/818 (94.87%)
iter = 786/818 (96.09%)
t = 3
iter = 10/609 (1.64%)
iter = 204/609 (33.50%)
iter = 243/609 (39.90%)
iter = 246/609 (40.39%)
iter = 294/609 (48.28%)
iter = 300/609 (49.26%)
iter = 370/609 (60.76%)
iter = 380/609 (62.40%)
iter = 382/609 (62.73%)
iter = 409/609 (67.16%)
iter = 549/609 (90.15%)
iter = 551/609 (90.48%)
t = 2
iter = 115/405 (28.40%)
iter = 131/405 (32.35%)
iter = 136/405 (33.58%)
iter = 158/405 (39.01%)
iter = 175/405 (43.21%)
iter = 182/405 (44.9

Let's try 2D Problem with another option and *additive* dynamics.

In [13]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 200
sol3 = solver.solve(pm2)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Precalculating points for value function evaluation: 1.0130 sec (CPU 1.0130 sec)
Computing value function in the last point: 0.0007 sec (CPU 0.0007 sec)
t = 4
iter = 907/5913 (15.34%)
iter = 979/5913 (16.56%)
iter = 1041/5913 (17.61%)
iter = 1508/5913 (25.50%)
iter = 1609/5913 (27.21%)
iter = 1646/5913 (27.84%)
iter = 1825/5913 (30.86%)
iter = 2417/5913 (40.88%)
iter = 3363/5913 (56.87%)
iter = 3484/5913 (58.92%)
iter = 3543/5913 (59.92%)
iter = 3603/5913 (60.93%)
iter = 4038/5913 (68.29%)
iter = 4153/5913 (70.24%)
iter = 4169/5913 (70.51%)
iter = 4219/5913 (71.35%)
iter = 4341/5913 (73.41%)
iter = 4506/5913 (76.20%)
iter = 4867/5913 (82.31%)
iter = 4996/5913 (84.49%)
iter = 5766/5913 (97.51%)
t = 3
iter = 125/3355 (3.73%)
iter = 182/3355 (5.42%)
iter = 507/3355 (15.11%)
iter = 1019/3355 (30.37%)
iter = 1180/3355 (35.17%)
iter = 1245/3355 (37.11%)
iter = 1432/3355 (42.68%)
iter = 1691/3355 (50.40%)
iter = 1700/3355 (50.67%)
iter = 1703/3355 (50.76%)
iter = 1777/3355 (52.97%)
iter = 202